# FFF Workshop

## B2: Sampling and scoring selections of catalogue compounds

### Outline

- Simulating a real-world selection problem
- Generating random selections
- Creating a Scorer object
- Comparing proposed Recipes

In [ ]:
import hippo
animal = hippo.HIPPO(
    "A71EV2A_demo",
    "../data/A71EV2A.sqlite",
)

## Simulating a real-world selection problem

We left the last notebook with a synthesis recipe describing how to get to 31 of our A71EV2A OpenBind scaffold compounds, which required almost €7k of reactants:

In [ ]:
chem_recipe = hippo.Recipe.from_json(animal.db, "c1_scaffolds_in_stock.json")

We also saw that tha average cost of our catalogue-available scaffold compounds was around €120, so it seems we might be able to come up with a more cost-effective plan using catalogue compounds instead of relying on in-house synthesis. Especially since some compounds are much cheaper still:

In [ ]:
quoted_scaffolds = animal.compounds(tag="openbind_a71ev2a_c1_scaffolds_quoted")
quoted_scaffolds[0].get_quotes(df=True)

What might not be totally obvious though is how we might quantify the quality of our selections. If we just wanted the most compounds for the given budget we could just sort the scaffolds by price and choose the top X until the budget is exceeded. First get an IngredientSet with 1mg amounts of every quoted scaffolds:

In [ ]:
quoted = quoted_scaffolds.as_ingredientset(amount=1)

Then let's get the cost of the cheapest 100:

In [ ]:
# get a DataFrame with prices (takes a moment as queries run)
df = quoted.price_df

# sort by ascending cost 
df = df.sort_values(by="price", ascending=True)

# get price of top 100 compounds
df.iloc[:100]["price"].sum()

## Generating random selections

HIPPO offers methods to generate both randomly sampling from a pool of [Routes](https://hippo-docs.winokan.com/en/latest/recipes.html#hippo.recipe.Route) (reaction networks to individual compounds) using the [RandomRecipeGenerator](https://hippo-docs.winokan.com/en/latest/sampling.html#hippo.rgen.RandomRecipeGenerator) object, or randomly sampling from a pool of [Compounds](https://hippo-docs.winokan.com/en/latest/compounds.html#hippo.compound.Compound) using the [RandomSelectionGenerator](https://hippo-docs.winokan.com/en/latest/sampling.html#hippo.rgen.RandomSelectionGenerator).

Both methods sample around a starting recipe/selection and while this is optional it is recommended as you can ensure a common core of no-brainer compounds in all the generated recipes. In our case let's look at the cost distribution in our scaffolds:

In [ ]:
df["price"].unique()

Let's say we wanted to order all compounds under €25 anyway, we could start with that selection:

In [ ]:
from hippo.price import Price
subset = df[df["price"] < Price(25, "EUR")]
subset["price"].sum()

Getting a CompoundSet for our selection:

In [ ]:
starting_selection = animal.compounds[subset["compound_id"]]
starting_selection

We can now create the RandomSelectionGenerator object:

In [ ]:
gen = hippo.rgen.RandomSelectionGenerator(
    db=animal.db,
    amount=1,                      # desired quantity for each compound in mg
    start_with=starting_selection, # common core for every selection
    compounds=quoted_scaffolds,    # the pool of scaffolds to select from
    quoted_only=True,              # only select compounds with available quotes
)

An output directory and serialised JSON representing the RSG have been created based on the database name. You can use the JSON to load the RandomSelectionGenerator object if you forgot the exact configuration:

In [ ]:
gen = hippo.rgen.RandomSelectionGenerator.from_json(animal.db, "A71EV2A_sgen.json")

Now that we are armed with our generator, we can start generating some selections:

In [ ]:
recipe = gen.generate(
    budget=7000,
    currency="EUR",
    max_iter=250, # cap the number of iterations for speed
)

recipe

If you're lucky this has given you far more than the 31 compounds than "c1_scaffolds_in_stock.json".

Let's generate some more recipes in a loop, this should take around 2 minutes:

In [ ]:
%%time
for i in range(50):
    gen.generate(budget=7000, max_iter=250)

## Creating a Scorer object

Now that we have at least 50 options to choose from in the `A71EV2A_selections` directory. We can create a [Scorer](https://hippo-docs.winokan.com/en/latest/sampling.html#hippo.scoring.Scorer) object to compare these options.

While any scorer with an arbitrary set of scoring metrics ([Attribute](https://hippo-docs.winokan.com/en/latest/sampling.html#hippo.scoring.Attribute), [CustomAttribute](https://hippo-docs.winokan.com/en/latest/sampling.html#hippo.scoring.CustomAttribute)) can be created, it's recommended to start with the [defaults](https://hippo-docs.winokan.com/en/latest/sampling.html#hippo.scoring.Scorer.default). These include:

- `num_compounds`: The number of product compounds in this selection. Higher is better.
- `num_inspirations`: The number of unique fragment compounds that inspired poses for product compounds in this selection. Higher is better.
- `num_inspiration_sets`: The number of unique fragment combinations that inspired poses for product compounds in this selection. Higher is better.
- `interaction_count`: The number of protein features that are being interecated with in this selection. Higher is better.
- `interaction_balance`: A measure for how evenly protein features are being interacted with in this selection using an h-index. Higher is better
- `num_subsites`: Count the number of subsites that poses in this set come into contact with. Higher is better.
- `subsite_balance`: Count the number of subsites that poses in this set come into contact with
- `avg_distance_score`: Average distance score (e.g. RMSD to fragment inspirations) for poses in this set. Lower is better.
- `avg_energy_score`: Average energy score (e.g. binding ddG) for poses in this set. Lower is better.
- `num_scaffolds`: The number of Syndirella scaffold compounds in this selection. Higher is better.
- `num_scaffolds_elaborated`: The number of Syndirella scaffold compounds that have at least one elaboration in this selection. Higher is better.
- `elaboration_balance`: A measure for how evenly scaffold compounds have been elaborated using an h-index. Higher is better.

N.B. that because we are not working with a full elaborated we will have to skip a lot of these.

In [ ]:
scorer = hippo.scoring.Scorer.default(
    db=animal.db,
    directory="A71EV2A_selections",
    skip=["num_scaffolds", "num_scaffolds_elaborated", "elaboration_balance", "subsite_balance", "num_subsites", "avg_distance_score", "avg_energy_score"],
    load_cache=False,
    
)

scorer

If you'd like to score for interactions these we'll need to profile the scaffold interactions first (this takes a few minutes):

In [ ]:
import mrich

poses = chem_recipe.poses + scorer.poses
display(poses)

for pose in mrich.track(poses):
    pose.calculate_interactions()

You can see all the active scoring metrics using the attributes property:

In [ ]:
scorer.attributes

You can customise their weights by setting them relative to each other:

In [ ]:
# sets all weight to be equal
scorer.weights = 1

# set custom weights
scorer.weights = [
    3,
    0,
    3,
    2,
    1,
]

The Scorer is _lazy_ meaning that attribute values and scores are only calculated when needed. This means that printing the summary may take a moment to update the scorer's cache when a large number of recipes are being scored. In this case it should still be quick:

In [ ]:
scorer.summary()

Notice that if you re-run the above cell it's near-instantaneous.

## Comparing proposed Recipes

The real fun begins when you start to plot and pick recipes based on the scores.

First let's add our benchmark Recipe using synthesis: `chem_recipe`

In [ ]:
scorer.add_recipes(["c1_scaffolds_in_stock.json"])
chem_recipe = scorer.recipes["c1_scaffolds_in_stock"]

Then we can look at how the number of compounds compare across the sets:

In [ ]:
scorer.plot(["price", "num_compounds"])

We can see that our `chem_recipe` is much worse than all our randomly generated selections.

We can get the best selection, or the top 5 selections like this:

In [ ]:
best = scorer.best
display(best)

top5 = scorer.top(5)
display(top5)

Let's look at our best selection in more detail:

In [ ]:
scorer.score(best, debug=True);

We can also compare to our chem_recipe:

In [ ]:
scorer.compare([best, chem_recipe])

`c1_scaffolds_in_stock` is very low in the `num_compounds` category, with only 31 compounds (in my data it was in the 2% percentile). But it might actually perform better in sampling the number of inspirations. This may just be due to which compounds are available from the catalogue. Suffice to say it is not always easy to determine the objectively best strategy.

You can look more closely at a specific attribute's distribution as follows:

In [ ]:
scorer.attributes[0].histogram()